In [1]:
import os
import pandas as pd
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

In [5]:
relative_path_to_new_data_destination = "../data/cleaned_jetblue_df.csv"

df = pd.read_csv(relative_path_to_new_data_destination)

In [6]:
df.head()

,searchDate,searchDayOfWeek,route,flightDate,flightDayOfWeek,startingAirport,destinationAirport,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsDurationInSeconds,segmentsCabinCode,departureTime,arrivalTime,departureCategory,arrivalCategory,isHolidaySearchDate,isHolidayFlightDate,nearHolidaySearchDate,nearHolidayFlightDate,searchDateInt,flightDateInt,daysLeft,numStops
0,2022-04-16,Saturday,ATL|EWR,2022-04-17,Sunday,ATL,EWR,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T07:30:00.000-04:00,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlueAirways,7740,coach,07:30:00,09:39:00,Morning,Morning,0,1,1,0,20220416,20220417,1,0
1,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T09:00:00.000-04:00,2022-04-17T11:15:00.000-04:00,JFK,ATL,JetBlueAirways,8100,coach,09:00:00,11:15:00,Morning,Noon,0,1,1,0,20220416,20220417,1,0
2,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T18:50:00.000-04:00,2022-04-17T21:13:00.000-04:00,JFK,ATL,JetBlueAirways,8580,coach,18:50:00,21:13:00,Afternoon,Evening,0,1,1,0,20220416,20220417,1,0
3,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,0,False,False,True,171.16,198.6,2,762.0,2022-04-17T12:03:00.000-04:00,2022-04-17T14:19:00.000-04:00,LGA,ATL,JetBlueAirways,8160,coach,12:03:00,14:19:00,Noon,Noon,0,1,1,0,20220416,20220417,1,0
4,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T10:40:00.000-04:00,2022-04-17T12:59:00.000-04:00,LGA,ATL,JetBlueAirways,8340,coach,10:40:00,12:59:00,Morning,Noon,0,1,1,0,20220416,20220417,1,0


In [32]:
checks = ["isHolidaySearchDate", "nearHolidaySearchDate"]

df[
    (df["isHolidaySearchDate"] == 0) &
    (df["nearHolidaySearchDate"] == 1)        
][checks]

,isHolidaySearchDate,nearHolidaySearchDate
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
3383595,0,1
3383596,0,1
3383597,0,1
3383598,0,1


In [9]:
df['searchDate'] = pd.to_datetime(df['searchDate'], format='%Y-%m-%d', errors='coerce')
df['flightDate'] = pd.to_datetime(df['flightDate'], format='%Y-%m-%d', errors='coerce')

In [10]:
df["searchDate"].min()

Timestamp('2022-04-16 00:00:00')

In [11]:
df["searchDate"].max()

Timestamp('2022-10-05 00:00:00')

In [12]:
df["flightDate"].min()

Timestamp('2022-04-17 00:00:00')

In [13]:
df["searchDate"].max()

Timestamp('2022-10-05 00:00:00')

In [32]:
df.head()

,searchDate,searchDayOfWeek,route,flightDate,flightDayOfWeek,startingAirport,destinationAirport,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsDurationInSeconds,segmentsCabinCode,departureTime,arrivalTime,departureCategory,arrivalCategory,searchDateInt,flightDateInt,daysLeft,numStops
0,2022-04-16,Saturday,ATL|EWR,2022-04-17,Sunday,ATL,EWR,PT2H9M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T07:30:00.000-04:00,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlueAirways,7740,coach,07:30:00,09:39:00,Morning,Morning,20220416,20220417,1,0
1,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,PT2H15M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T09:00:00.000-04:00,2022-04-17T11:15:00.000-04:00,JFK,ATL,JetBlueAirways,8100,coach,09:00:00,11:15:00,Morning,Noon,20220416,20220417,1,0
2,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,PT2H23M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T18:50:00.000-04:00,2022-04-17T21:13:00.000-04:00,JFK,ATL,JetBlueAirways,8580,coach,18:50:00,21:13:00,Afternoon,Evening,20220416,20220417,1,0
3,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,PT2H16M,0,False,False,True,171.16,198.6,2,762.0,2022-04-17T12:03:00.000-04:00,2022-04-17T14:19:00.000-04:00,LGA,ATL,JetBlueAirways,8160,coach,12:03:00,14:19:00,Noon,Noon,20220416,20220417,1,0
4,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,PT2H19M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T10:40:00.000-04:00,2022-04-17T12:59:00.000-04:00,LGA,ATL,JetBlueAirways,8340,coach,10:40:00,12:59:00,Morning,Noon,20220416,20220417,1,0


In [33]:
df.duplicated().sum()

0

In [10]:
df.dtypes

searchDate                       object
searchDayOfWeek                  object
route                            object
flightDate                       object
flightDayOfWeek                  object
startingAirport                  object
destinationAirport               object
travelDuration                   object
elapsedDays                       int64
isBasicEconomy                     bool
isRefundable                       bool
isNonStop                          bool
baseFare                        float64
totalFare                       float64
seatsRemaining                    int64
totalTravelDistance             float64
segmentsDepartureTimeRaw         object
segmentsArrivalTimeRaw           object
segmentsArrivalAirportCode       object
segmentsDepartureAirportCode     object
segmentsAirlineName              object
segmentsDurationInSeconds        object
segmentsCabinCode                object
departureTime                    object
arrivalTime                      object


In [26]:
cols_to_strip_white_space = [
    "route",
    "startingAirport",
    "destinationAirport",
    "segmentsArrivalAirportCode",
    "segmentsDepartureAirportCode",
    "segmentsCabinCode",
    "segmentsAirlineName"
]

for col in cols_to_strip_white_space:
    df[col] = df[col].str.replace(r"\s+", "", regex=True)

In [27]:
df.head()

,searchDate,searchDayOfWeek,route,flightDate,flightDayOfWeek,startingAirport,destinationAirport,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsDurationInSeconds,segmentsCabinCode,departureTime,arrivalTime,departureCategory,arrivalCategory,searchDateInt,flightDateInt,daysLeft,numStops
0,2022-04-16,Saturday,ATL|EWR,2022-04-17,Sunday,ATL,EWR,PT2H9M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T07:30:00.000-04:00,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlueAirways,7740,coach,07:30:00,09:39:00,Morning,Morning,20220416,20220417,1,0
1,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,PT2H15M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T09:00:00.000-04:00,2022-04-17T11:15:00.000-04:00,JFK,ATL,JetBlueAirways,8100,coach,09:00:00,11:15:00,Morning,Noon,20220416,20220417,1,0
2,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,PT2H23M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T18:50:00.000-04:00,2022-04-17T21:13:00.000-04:00,JFK,ATL,JetBlueAirways,8580,coach,18:50:00,21:13:00,Afternoon,Evening,20220416,20220417,1,0
3,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,PT2H16M,0,False,False,True,171.16,198.6,2,762.0,2022-04-17T12:03:00.000-04:00,2022-04-17T14:19:00.000-04:00,LGA,ATL,JetBlueAirways,8160,coach,12:03:00,14:19:00,Noon,Noon,20220416,20220417,1,0
4,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,PT2H19M,0,False,False,True,171.16,198.6,7,762.0,2022-04-17T10:40:00.000-04:00,2022-04-17T12:59:00.000-04:00,LGA,ATL,JetBlueAirways,8340,coach,10:40:00,12:59:00,Morning,Noon,20220416,20220417,1,0


In [25]:
df = df.apply(lambda col: col.str.strip() for col in cols_to_strip_white_space)

ValueError: No objects to concatenate

In [23]:
df["segmentsAirlineName"].value_counts()

segmentsAirlineName
JetBlue Airways                                      2815042
JetBlue Airways||JetBlue Airways                     1125381
JetBlue Airways||JetBlue Airways||JetBlue Airways       4618
Cape Air||JetBlue Airways                               1842
JetBlue Airways||Cape Air                               1353
American Airlines||JetBlue Airways                       388
JetBlue Airways||Cape Air||Cape Air                      254
Cape Air||Cape Air||JetBlue Airways                      248
JetBlue Airways||JetBlue Airways||Cape Air               125
Cape Air||JetBlue Airways||JetBlue Airways               105
JetBlue Airways||Silver Airways||JetBlue Airways          80
JetBlue Airways||Cape Air||JetBlue Airways                30
JetBlue Airways||American Airlines                        13
Cape Air||Cape Air||Cape Air||JetBlue Airways              1
Name: count, dtype: int64

In [12]:
df["segmentsAirlineName"].value_counts()

segmentsAirlineName
JetBlue Airways                                      2815042
JetBlue Airways||JetBlue Airways                     1125381
JetBlue Airways||JetBlue Airways||JetBlue Airways       4618
Cape Air||JetBlue Airways                               1842
JetBlue Airways||Cape Air                               1353
American Airlines||JetBlue Airways                       388
JetBlue Airways||Cape Air||Cape Air                      254
Cape Air||Cape Air||JetBlue Airways                      248
JetBlue Airways||JetBlue Airways||Cape Air               125
Cape Air||JetBlue Airways||JetBlue Airways               105
JetBlue Airways||Silver Airways||JetBlue Airways          80
JetBlue Airways||Cape Air||JetBlue Airways                30
JetBlue Airways||American Airlines                        13
Cape Air||Cape Air||Cape Air||JetBlue Airways              1
Name: count, dtype: int64

In [36]:
df[df.duplicated()]

,searchDate,searchDayOfWeek,route,flightDate,flightDayOfWeek,startingAirport,destinationAirport,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsDurationInSeconds,segmentsCabinCode,departureTime,arrivalTime,departureCategory,arrivalCategory,searchDateInt,flightDateInt,daysLeft,numStops
1557,2022-04-16,Saturday,JFK|LAX,2022-04-22,Friday,JFK,LAX,PT6H22M,0,False,False,True,222.33,253.61,7,2458.0,2022-04-22T18:30:00.000-04:00,2022-04-22T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220416,20220422,6,0
4326,2022-04-17,Sunday,JFK|LAX,2022-04-22,Friday,JFK,LAX,PT6H22M,0,False,False,True,222.33,253.61,7,2458.0,2022-04-22T18:30:00.000-04:00,2022-04-22T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220417,20220422,5,0
30395,2022-04-18,Monday,JFK|LAX,2022-04-22,Friday,JFK,LAX,PT6H22M,0,False,False,True,222.33,253.61,7,2458.0,2022-04-22T18:30:00.000-04:00,2022-04-22T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220418,20220422,4,0
52229,2022-04-19,Tuesday,JFK|LAX,2022-04-22,Friday,JFK,LAX,PT6H22M,0,False,False,True,222.33,253.61,7,2458.0,2022-04-22T18:30:00.000-04:00,2022-04-22T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220419,20220422,3,0
78463,2022-04-20,Wednesday,JFK|LAX,2022-04-22,Friday,JFK,LAX,PT6H22M,0,False,False,True,322.79,361.60,7,2458.0,2022-04-22T18:30:00.000-04:00,2022-04-22T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220420,20220422,2,0
94351,2022-04-21,Thursday,JFK|LAX,2022-04-22,Friday,JFK,LAX,PT6H22M,0,False,False,True,322.79,361.60,7,2458.0,2022-04-22T18:30:00.000-04:00,2022-04-22T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220421,20220422,1,0
121300,2022-04-22,Friday,JFK|LAX,2022-04-23,Saturday,JFK,LAX,PT6H22M,0,False,False,True,322.79,361.60,7,2458.0,2022-04-23T18:30:00.000-04:00,2022-04-23T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220422,20220423,1,0
332368,2022-05-01,Sunday,JFK|LAX,2022-05-02,Monday,JFK,LAX,PT6H22M,0,False,False,True,322.79,361.60,7,2458.0,2022-05-02T18:30:00.000-04:00,2022-05-02T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220501,20220502,1,0
358893,2022-05-02,Monday,JFK|LAX,2022-05-03,Tuesday,JFK,LAX,PT6H22M,0,False,False,True,322.79,361.60,7,2458.0,2022-05-03T18:30:00.000-04:00,2022-05-03T21:52:00.000-07:00,LAX,JFK,JetBlue Airways,22920,coach,18:30:00,21:52:00,Afternoon,Evening,20220502,20220503,1,0


In [24]:
df.columns

Index(['searchDate', 'searchDayOfWeek', 'route', 'flightDate',
       'flightDayOfWeek', 'startingAirport', 'destinationAirport',
       'travelDuration', 'elapsedDays', 'isBasicEconomy', 'isRefundable',
       'isNonStop', 'baseFare', 'totalFare', 'seatsRemaining',
       'totalTravelDistance', 'segmentsDepartureTimeRaw',
       'segmentsArrivalTimeRaw', 'segmentsArrivalAirportCode',
       'segmentsDepartureAirportCode', 'segmentsAirlineName',
       'segmentsDurationInSeconds', 'segmentsCabinCode', 'departureTime',
       'arrivalTime', 'departureCategory', 'arrivalCategory', 'searchDateInt',
       'flightDateInt', 'daysLeft', 'numStops'],
      dtype='object')

In [25]:
sorted(df.columns)

['arrivalCategory',
 'arrivalTime',
 'baseFare',
 'daysLeft',
 'departureCategory',
 'departureTime',
 'destinationAirport',
 'elapsedDays',
 'flightDate',
 'flightDateInt',
 'flightDayOfWeek',
 'isBasicEconomy',
 'isNonStop',
 'isRefundable',
 'numStops',
 'route',
 'searchDate',
 'searchDateInt',
 'searchDayOfWeek',
 'seatsRemaining',
 'segmentsAirlineName',
 'segmentsArrivalAirportCode',
 'segmentsArrivalTimeRaw',
 'segmentsCabinCode',
 'segmentsDepartureAirportCode',
 'segmentsDepartureTimeRaw',
 'segmentsDurationInSeconds',
 'startingAirport',
 'totalFare',
 'totalTravelDistance',
 'travelDuration']

In [26]:
df["searchDayOfWeek"].dtype

dtype('O')

In [ ]:
import os
import pandas as pd
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt


##Functions and 
#Datetime Conversion
def table_datetime_conversion(df):
    # Convert 'searchDate' and 'flightDate' to datetime, handling potential errors
    df['searchDate'] = pd.to_datetime(df['searchDate'], format='%Y-%m-%d', errors='coerce')
    df['flightDate'] = pd.to_datetime(df['flightDate'], format='%Y-%m-%d', errors='coerce')

    # Ensure the columns are of datetime type
    df['searchDate'] = df['searchDate'].astype('datetime64[ns]')
    df['flightDate'] = df['flightDate'].astype('datetime64[ns]')

    return df


#For start-end time classification 
def extract_time(time_string):
    segments = time_string.split("||")
    first_segment = segments[0]

    datetime = dt.fromisoformat(first_segment)
    return datetime.time()

#For start-end time classification 
def time_classification(time):
    hour=time.hour #get the hour from the time object
    #ex: 06:30 -> 6
    if hour in range(3,7): #03:00 - 06:00
        return 'Early Morning'
    elif hour in range(7,11):
        return 'Morning'
    elif hour in range(11,15):
        return 'Noon'
    elif hour in range(15,19):
        return 'Afternoon'
    elif hour in range(19,23):
        return 'Evening'
    elif hour in [23,0,1,2]: # 23:00 - 02:00 (covering late night)
        return 'Late night'
    else:
        return 'Unknown'


#For non-KPI feature creation
def feature_creation(df):
    
    #Extracted Time Columns
    df['departureTime']=df['segmentsDepartureTimeRaw'].apply(extract_time)
    df['arrivalTime']=df['segmentsArrivalTimeRaw'].apply(extract_time)

    #Categorizing ordinal times
    df['departureCategory'] = df['departureTime'].apply(time_classification)
    df['arrivalCategory'] = df['arrivalTime'].apply(time_classification)

    #Route
    df['route']=df['startingAirport'] + '|' + df['destinationAirport']
    relocate_route_col = df.pop('route')
    df.insert(1,'route', relocate_route_col)
    
    return df



#For KPI feature creation
def kpi_creation(df):

    df["daysLeft"] = (df["flightDate"] - df["searchDate"]).dt.days
    df['numStops']= df['segmentsAirlineName'].apply(lambda x: len(x.split('||'))-1)
    # df["dollarPerMile"]
    # df["milePerDollar"]
    # df["monthlyAverageRoutePrice"]

    return df



### Table Transformation# ###

##Bring in table
# relative_path_to_native_data = "../data/jetblue_df.csv"
# relative_path_to_new_data_destination = "../data/cleaned_jetblue_df.csv"
# relative_path_to_new_data_destination = "../data/testing_cleaned_jetblue_df.csv"

df = pd.read_csv(relative_path_to_native_data) 




##Drop unused columns
dropped_columns = [
    "Unnamed: 0", #useless
    "segmentsAirlineCode", #redundant
    "segmentsEquipmentDescription", #dirty
    "fareBasisCode", #bloat
    "legId", #bloat
    "segmentsDistance", #bloat
    # "travelDuration" #bloat?
    "segmentsDepartureTimeEpochSeconds",
    "segmentsArrivalTimeEpochSeconds"

]
df.drop(columns=dropped_columns, axis=1, inplace=True)


##Remove Null Values
df.dropna(inplace=True)


#Filtering

# Filtering only "NY" routes..
# EWR (NJ airport) included as it's poopular for New Yorkers to travel to Newark for flights.
ny_filter = ["JFK", "LGA", "EWR"]
df = df[
    (df["startingAirport"].isin(ny_filter)) |         #only starting airport
    (df["destinationAirport"].isin(ny_filter))        #only destination airports
    ]  


# ##iltering out JetBlue partners
# pure_jetblue_filter = ['JetBlue Airways', 'JetBlue Airways||JetBlue Airways','JetBlue Airways||JetBlue Airways||JetBlue Airways'] #here for interpretability
# df = df[ df["segmentsAirlineName"].isin(pure_jetblue_filter) ]



##Label cleaning
#Datetime Conversion
df = table_datetime_conversion(df)


##Feature Engineering
#Feature Creation
df = feature_creation(df)


#KPI Creation
df = kpi_creation(df)

In [16]:
df.head()

,searchDate,searchDayOfWeek,route,flightDate,flightDayOfWeek,startingAirport,destinationAirport,travelDuration,elapsedDays,isBasicEconomy,...,segmentsDepartureAirportCode,segmentsAirlineName,segmentsDurationInSeconds,segmentsCabinCode,departureTime,arrivalTime,departureCategory,arrivalCategory,daysLeft,numStops
0,2022-04-16,Saturday,ATL|EWR,2022-04-17,Sunday,ATL,EWR,PT2H9M,0,False,...,ATL,JetBlue Airways,7740,coach,07:30:00,09:39:00,Morning,Morning,1,0
1,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,PT2H15M,0,False,...,ATL,JetBlue Airways,8100,coach,09:00:00,11:15:00,Morning,Noon,1,0
2,2022-04-16,Saturday,ATL|JFK,2022-04-17,Sunday,ATL,JFK,PT2H23M,0,False,...,ATL,JetBlue Airways,8580,coach,18:50:00,21:13:00,Afternoon,Evening,1,0
3,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,PT2H16M,0,False,...,ATL,JetBlue Airways,8160,coach,12:03:00,14:19:00,Noon,Noon,1,0
4,2022-04-16,Saturday,ATL|LGA,2022-04-17,Sunday,ATL,LGA,PT2H19M,0,False,...,ATL,JetBlue Airways,8340,coach,10:40:00,12:59:00,Morning,Noon,1,0


In [74]:
print(df.shape)
print(df.size)

(3949480, 27)
106635960


In [75]:
df["segmentsAirlineName"].value_counts()

segmentsAirlineName
JetBlue Airways                                      2815042
JetBlue Airways||JetBlue Airways                     1125381
JetBlue Airways||JetBlue Airways||JetBlue Airways       4618
Cape Air||JetBlue Airways                               1842
JetBlue Airways||Cape Air                               1353
American Airlines||JetBlue Airways                       388
JetBlue Airways||Cape Air||Cape Air                      254
Cape Air||Cape Air||JetBlue Airways                      248
JetBlue Airways||JetBlue Airways||Cape Air               125
Cape Air||JetBlue Airways||JetBlue Airways               105
JetBlue Airways||Silver Airways||JetBlue Airways          80
JetBlue Airways||Cape Air||JetBlue Airways                30
JetBlue Airways||American Airlines                        13
Cape Air||Cape Air||Cape Air||JetBlue Airways              1
Name: count, dtype: int64

In [66]:
df["startingAirport"].value_counts()

startingAirport
LGA    856414
JFK    751485
EWR    321875
MIA    250128
CLT    247767
LAX    242154
BOS    239770
ORD    218292
DFW    201869
SFO    180401
PHL    160878
DTW    114638
ATL     96556
DEN     62814
Name: count, dtype: int64

In [67]:
df["destinationAirport"].value_counts()

destinationAirport
LGA    928804
JFK    753057
EWR    333409
MIA    264245
BOS    240478
CLT    238337
LAX    233104
ORD    224279
DFW    179712
SFO    171185
PHL    130145
DTW    106404
ATL     86897
DEN     54985
Name: count, dtype: int64

In [ ]:
rows_before_ny_filter = 6768004
rows_after_ny_filter = 3949480


size_before_ny_filter = 135360080
size_after_ny_filter = 78989600

row_difference = rows_before_ny_filter - rows_after_ny_filter

size_difference = size_before_ny_filter - size_after_ny_filter

print(row_difference)
print(size_difference)

2818524
56370480


In [62]:
print(df.shape)
print(df.size)

(3945041, 27)
106516107


In [61]:
rows_before_airline_filter = 3949480
rows_after_airline_filter = 3945041


size_before_airline_filter = 78989600
size_after_airline_filter = 78900820

row_difference = rows_before_airline_filter - rows_after_airline_filter

size_difference = size_before_airline_filter - size_after_airline_filter

print(row_difference)
print(size_difference)

4439
88780


In [78]:
relative_path_to_new_data_destination = "../data/cleaned_jetblue_df.csv"

df = pd.read_csv(relative_path_to_new_data_destination)

df.head()

,searchDate,route,flightDate,startingAirport,destinationAirport,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,...,segmentsDepartureAirportCode,segmentsAirlineName,segmentsDurationInSeconds,segmentsCabinCode,departureTime,arrivalTime,departureCategory,arrivalCategory,daysLeft,numStops
0,2022-04-16,ATL|EWR,2022-04-17,ATL,EWR,PT2H9M,0,False,False,True,...,ATL,JetBlue Airways,7740,coach,07:30:00,09:39:00,Morning,Morning,1,0
1,2022-04-16,ATL|JFK,2022-04-17,ATL,JFK,PT2H15M,0,False,False,True,...,ATL,JetBlue Airways,8100,coach,09:00:00,11:15:00,Morning,Noon,1,0
2,2022-04-16,ATL|JFK,2022-04-17,ATL,JFK,PT2H23M,0,False,False,True,...,ATL,JetBlue Airways,8580,coach,18:50:00,21:13:00,Afternoon,Evening,1,0
3,2022-04-16,ATL|LGA,2022-04-17,ATL,LGA,PT2H16M,0,False,False,True,...,ATL,JetBlue Airways,8160,coach,12:03:00,14:19:00,Noon,Noon,1,0
4,2022-04-16,ATL|LGA,2022-04-17,ATL,LGA,PT2H19M,0,False,False,True,...,ATL,JetBlue Airways,8340,coach,10:40:00,12:59:00,Morning,Noon,1,0


In [79]:
df.shape

(3949480, 27)